In [2]:
pip install pandas requests 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd

In [4]:
home_url = 'https://api.mfapi.in/mf'
r = requests.get(home_url)
r.json()

[{'schemeCode': 100027,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF-Half Yearly Dividend'},
 {'schemeCode': 100028,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF-Quaterly Dividend'},
 {'schemeCode': 100029,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF-Growth'},
 {'schemeCode': 100030,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF-Annual Dividend'},
 {'schemeCode': 100031,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF - ST-Dividend'},
 {'schemeCode': 100032,
  'schemeName': 'Grindlays Super Saver Income Fund-GSSIF - ST-Growth'},
 {'schemeCode': 100033,
  'schemeName': 'Aditya Birla Sun Life Equity Advantage Fund - Regular Growth'},
 {'schemeCode': 100034,
  'schemeName': 'Aditya Birla Sun Life Equity Advantage Fund -Regular Dividend'},
 {'schemeCode': 100034,
  'schemeName': 'Aditya Birla Sun Life Equity Advantage Fund -Regular - IDCW'},
 {'schemeCode': 100035,
  'schemeName': 'Birla Sun Life Freedom Fund-Plan A (Dividend)'},
 {'

In [5]:
scheme_df = pd.DataFrame.from_records(r.json())
scheme_df

,schemeCode,schemeName
0,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...
1,100028,Grindlays Super Saver Income Fund-GSSIF-Quater...
2,100029,Grindlays Super Saver Income Fund-GSSIF-Growth
3,100030,Grindlays Super Saver Income Fund-GSSIF-Annual...
4,100031,Grindlays Super Saver Income Fund-GSSIF - ST-D...
...,...,...
47001,152901,Baroda BNP Paribas Credit Risk fund -Regular -...
47002,152902,Baroda BNP Paribas Credit Risk fund -Regular -...
47003,152903,Baroda BNP Paribas Credit Risk fund - Regular-...
47004,152904,Baroda BNP Paribas Credit Risk fund -Regular-G...


In [6]:
scheme_id = scheme_df.loc[245]['schemeCode']
scheme_url = "https://api.mfapi.in/mf/{0}"


In [7]:
r2 = requests.get(scheme_url.format(scheme_id))
r2.json()

{'meta': {'fund_house': 'LIC Mutual Fund',
  'scheme_type': 'Open Ended Schemes',
  'scheme_category': 'Hybrid Scheme - Aggressive Hybrid Fund',
  'scheme_code': 100323,
  'scheme_name': 'LIC MF Aggressive Hybrid Fund-Regular Plan-Growth'},
 'data': [{'date': '13-09-2024', 'nav': '198.10190'},
  {'date': '12-09-2024', 'nav': '197.89080'},
  {'date': '11-09-2024', 'nav': '195.72030'},
  {'date': '10-09-2024', 'nav': '196.11940'},
  {'date': '09-09-2024', 'nav': '194.32650'},
  {'date': '06-09-2024', 'nav': '194.88540'},
  {'date': '05-09-2024', 'nav': '196.68810'},
  {'date': '04-09-2024', 'nav': '196.13620'},
  {'date': '03-09-2024', 'nav': '195.76150'},
  {'date': '02-09-2024', 'nav': '195.71760'},
  {'date': '30-08-2024', 'nav': '196.14230'},
  {'date': '29-08-2024', 'nav': '195.53400'},
  {'date': '28-08-2024', 'nav': '195.57490'},
  {'date': '27-08-2024', 'nav': '195.47930'},
  {'date': '26-08-2024', 'nav': '195.36570'},
  {'date': '23-08-2024', 'nav': '194.74580'},
  {'date': '22-

In [8]:
# Load all the data to dataframe (for now 10 schemes data)
scheme_pricing = pd.DataFrame()
for i in range(10):
    scode = scheme_df.loc[i]['schemeCode']
    r2 = requests.get(scheme_url.format(scode))
    if r2.status_code == 200:
        resp_json = r2.json()
        temp_df = pd.DataFrame.from_records(resp_json['data'])
        temp_df['schemeCode'] = scode
        temp_df['fundHouse'] = resp_json['meta']['fund_house']
        temp_df['schemeType'] = resp_json['meta']['scheme_type']
        temp_df['schemeCategory'] = resp_json['meta']['scheme_category']
        temp_df['schemeName'] = resp_json['meta']['scheme_name']
        scheme_pricing = pd.concat([scheme_pricing, temp_df])
    else:
        print(f"Warning: Invalid HTTP status code {r2.status_code}")

In [9]:
scheme_pricing.count()

date              18183
nav               18183
schemeCode        18183
fundHouse         18183
schemeType        18183
schemeCategory    18183
schemeName        18183
dtype: int64

In [10]:
scheme_pricing.head()

,date,nav,schemeCode,fundHouse,schemeType,schemeCategory,schemeName
0,29-05-2008,10.72050,100027,Standard Chartered Mutual Fund,Open Ended Schemes,Income,Grindlays Super Saver Income Fund-GSSIF-Half Y...
1,28-05-2008,10.72500,100027,Standard Chartered Mutual Fund,Open Ended Schemes,Income,Grindlays Super Saver Income Fund-GSSIF-Half Y...
2,27-05-2008,10.72160,100027,Standard Chartered Mutual Fund,Open Ended Schemes,Income,Grindlays Super Saver Income Fund-GSSIF-Half Y...
3,26-05-2008,10.72060,100027,Standard Chartered Mutual Fund,Open Ended Schemes,Income,Grindlays Super Saver Income Fund-GSSIF-Half Y...
4,23-05-2008,10.71520,100027,Standard Chartered Mutual Fund,Open Ended Schemes,Income,Grindlays Super Saver Income Fund-GSSIF-Half Y...


In [21]:
scheme_pricing['date'] = pd.to_datetime(scheme_pricing['date'], dayfirst=True)
scheme_pricing =  scheme_pricing.set_index('date')

In [12]:
pip install plotly==5.24.1 nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [13]:
import plotly.express as px

In [22]:
scheme_pricing.loc["2009"]

,nav,schemeCode,fundHouse,schemeType,schemeCategory,schemeName
date,,,,,,
2009-12-31,151.66000,100033,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...
2009-12-30,151.07000,100033,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...
2009-12-29,151.32000,100033,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...
2009-12-24,151.01000,100033,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...
2009-12-23,150.07000,100033,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...
...,...,...,...,...,...,...
2009-01-07,17.30000,100035,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Income,Birla Sun Life Freedom Fund-Plan A (Dividend)
2009-01-06,17.97000,100035,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Income,Birla Sun Life Freedom Fund-Plan A (Dividend)
2009-01-05,17.96000,100035,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Income,Birla Sun Life Freedom Fund-Plan A (Dividend)


In [ ]:
scheme_pricing.reset_index(inplace=True)

In [55]:

filtered_df = scheme_pricing[(scheme_pricing['schemeCode'] == 100035)]
filtered_df
fig = px.bar(filtered_df,  y='nav', x='date', title='nav')
fig.show()